In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [4]:
import pandas as pd
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

from src.bert.dataset import BERTClassificationDataModule
from src.bert.model import BERTClassificationModel

In [5]:
DATA_PATH = "../data/sentiment_data.csv"

In [6]:
data = pd.read_csv(DATA_PATH)

In [7]:
data.SentimentScore -= 1

In [8]:
data

,issuerid,MessageText,SentimentScore
0,153,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,1
1,230,Ozon продолжает развивать специализированные ф...,3
2,118,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,3
3,220,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,4
4,89,​​Windfall Tax — налог на сверхприбыль. Какие ...,1
...,...,...,...
9316,157,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,3
9317,157,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,3
9318,225,​​Ключевой принцип создания портфеля 🔹Диверси...,2
9319,127,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",2


In [9]:
train_df, test_df = train_test_split(data, test_size=0.25, random_state=42)

In [10]:
MODEL_NAME = "cointegrated/rubert-tiny"
BATCH_SIZE = 64
NUM_EPOCHS = 5

In [11]:
num_issuers_classes = train_df.issuerid.max() + 1
num_sentiment_classes = train_df.SentimentScore.nunique()
num_issuers_classes, num_sentiment_classes

(275, 5)

In [12]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

In [13]:
data_module = BERTClassificationDataModule(train_df=train_df,
                                           test_df=test_df,
                                           tokenizer=tokenizer,
                                           num_issuers_classes=num_issuers_classes,
                                           num_sentiment_classes=num_sentiment_classes,
                                           batch_size=BATCH_SIZE)
data_module.setup()

In [14]:
model = BERTClassificationModel(model_name=MODEL_NAME,
                                num_issuers_classes=num_issuers_classes,
                                num_sentiment_classes=num_sentiment_classes)

In [15]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="sentiment",
    save_top_k=1,
    verbose=True,
    monitor="loss/val",
    mode="min",
)

trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=NUM_EPOCHS,
    accelerator="cuda",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
!rm -r lightning_logs

rm: cannot remove 'lightning_logs': No such file or directory


In [17]:
trainer.fit(model, data_module)

You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/worker/workspace/hakaton-gagarin-sentiment_interface/pybooks/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [5]
/home/worker/workspace/ctc-ocr-torch/venv/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name           | Type       | Params
----------------------------------------------
0 | bert           | BertModel  | 11.8 M
1 | issuers_head   | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 87: 'loss/val' reached 0.39289 (best 0.39289), saving model to '/home/worker/workspace/hakaton-gagarin-sentiment_interface/pybooks/checkpoints/sentiment.ckpt' as top 1


...

Epoch 2, global step 261: 'loss/val' reached 0.37677 (best 0.37677), saving model to '/home/worker/workspace/hakaton-gagarin-sentiment_interface/pybooks/checkpoints/sentiment.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 3, global step 348: 'loss/val' reached 0.37033 (best 0.37033), saving model to '/home/worker/workspace/hakaton-gagarin-sentiment_interface/pybooks/checkpoints/sentiment.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 4, global step 435: 'loss/val' reached 0.37012 (best 0.37012), saving model to '/home/worker/workspace/hakaton-gagarin-sentiment_interface/pybooks/checkpoints/sentiment.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=5` reached.


In [18]:
trained_model = BERTClassificationModel.load_from_checkpoint("checkpoints/sentiment.ckpt",
                                                             model_name=MODEL_NAME,
                                                             num_issuers_classes=num_issuers_classes,
                                                             num_sentiment_classes=num_sentiment_classes)

In [19]:
from src.bert.utils import evaluate_metric, generate_answer_batched

In [20]:
issuer_preds, sentiment_preds = generate_answer_batched(trained_model, tokenizer, test_df)

  0%|          | 0/37 [00:00<?, ?it/s]

In [22]:
# [pin]

evaluate_metric(
    company_predictions=issuer_preds,
    company_labels=test_df.issuerid.tolist(),
    sentiment_predictions=sentiment_preds,
    sentiment_labels=test_df.SentimentScore.tolist(), 
)

{'total': 30.74869398162147,
 'f1': 0.03711030176885155,
 'accuracy': 0.5778635778635779}